In [1]:
from typing import Tuple, MutableMapping, Set, Sequence, List

import clingo
from clingo import Propagator, PropagateInit, PropagateControl

In [2]:
inst1 = """

node(1).
node(2).
node(3).

edge((1,2)).
edge((1,3)).
edge((2,1)).
edge((2,3)).
edge((3,1)).
edge((3,2)).

edge_weight((1,2), 10).
edge_weight((2,3), 10).
edge_weight((3,1), 10).
edge_weight((2,1), 100).
edge_weight((3,2), 100).
edge_weight((1,3), 100).

mintime(1).
maxtime(6).

"""

In [3]:
inst2 = """

node(1).
node(2).
node(3).
node(4).

edge((1,2)).
edge((1,3)).
edge((1,4)).
edge((2,1)).
edge((2,3)).
edge((3,1)).
edge((3,2)).
edge((3,4)).
edge((4,1)).
edge((4,3)).

edge_weight((1,2), 10).
edge_weight((2,3), 10).
edge_weight((3,1), 10).
edge_weight((3,4), 10).
edge_weight((4,1), 10).
edge_weight((1,4), 100).
edge_weight((2,1), 100).
edge_weight((3,2), 100).
edge_weight((1,3), 100).
edge_weight((4,3), 100).

mintime(1).
maxtime(10).

"""

In [4]:
reasoning = """

time(I..A) :- mintime(I), maxtime(A).

lastwalk(L) :- walk(_,L), not walk(_,L+1).

%1 { walk(E,I)   : edge(E) } 1 :- mintime(I).
%1 { walk(E,I+1) : edge(E) } 1 :- mintime(I).
%  { walk(E,T)   : edge(E) } 1 :- walk(_,T-1), maxtime(A), T<=A.

0 { walk(E,T) : edge(E) } 1 :- time(T).

:- mintime(I), not walk((1,_), I).
:- lastwalk(L), not walk((_,1), L).
%:- edge((U,V)), edge((V,U)), not walk((U,V), _), not walk((V,U), _).

#show walk/2.
"""

In [5]:
class WindyPropagator(Propagator):

    def __init__(self):
        self._e2l: MutableMapping[Tuple[int, int], int] = {}  # {(int,int): literal}
        self._l2e: MutableMapping[int, Tuple[int, int]] = {}  # {literal: (int,int)}
        self._walks: MutableMapping[int, Set[int]] = {}  # {literal: {literal}}
        self._incidents: MutableMapping[int, Set[int]] = {}  # {literal: {literal}}
        self._w2s : MutableMapping[int, int] = {}
        self._s2w : MutableMapping[int, int] = {}
        self._decision_stack : List[Tuple[int,int]] = []

    @staticmethod
    def symbolic_atom_to_edge(edge: clingo.SymbolicAtom) -> Tuple[int, int]:
        return WindyPropagator.symbol_to_edge(edge.symbol)

    @staticmethod
    def symbol_to_edge(edge: clingo.Symbol) -> Tuple[int, int]:
        u = edge.arguments[0].arguments[0].number
        v = edge.arguments[0].arguments[1].number
        return u, v

    def __sol_lit_to_edge_lit(self, sol_lit: int) -> int:
        return self.__walk_lit_to_edge_lit(self._s2w[sol_lit])

    def __walk_lit_to_edge_lit(self, walk_lit: int) -> int:
        for (edge_lit, walk_lits) in self._walks.items():
            if walk_lit in walk_lits:
                return edge_lit
        assert False, "Should have found walk literal {}".format(walk_lit)

    def __sol_lit_to_edge(self, sol_lit: int) -> Tuple[int, int]:
        return self._l2e[self.__sol_lit_to_edge_lit(sol_lit)]

    def __walk_lit_to_edge(self, walk_lit: int) -> Tuple[int, int]:
        return self._l2e[self.__walk_lit_to_edge_lit(walk_lit)]

    def init(self, init: PropagateInit) -> None:
        print("[DEBUG]:", "Initializing Propagator")
        edges = init.symbolic_atoms.by_signature('edge', 1)
        print("[DEBUG]:", "Building edge dict")
        for edge in edges:
            u, v = WindyPropagator.symbolic_atom_to_edge(edge)
            edge_lit = edge.literal
            self._e2l[(u, v)] = edge_lit
            self._l2e[edge_lit] = (u, v)
            walks = init.symbolic_atoms.by_signature('walk', 2)
            for walk in walks:
                walk_symbol = walk.symbol
                walk_edge = walk_symbol.arguments[0]
                walk_from = walk_edge.arguments[0].number
                walk_to = walk_edge.arguments[1].number
                if walk_from == u and walk_to == v:
                    walk_lit = walk.literal
                    self._walks.setdefault(edge_lit, set()).add(walk_lit)
        walks = init.symbolic_atoms.by_signature('walk', 2)
        print("[DEBUG]:", "Building walks dict")
        for walk in walks:
            self._w2s[walk.literal] = init.solver_literal(walk.literal)
            self._s2w[self._w2s[walk.literal]] = walk.literal
            init.add_watch(self._w2s[walk.literal])

        print("[DEBUG]:", "Building incidents dict")
        for (edge_lit1, (u, v)) in self._l2e.items():
            for (edge_lit2, (w, x)) in self._l2e.items():
                if v == w:
                    self._incidents.setdefault(edge_lit1, set()).add(edge_lit2)

        print("[DEBUG]:", "Adding walk clauses")
        for (edge_lit, walk_lits) in self._walks.items():
            print("[DEBUG]:", "Adding clause of {} literals for {}".format(len(walk_lits), self._l2e[edge_lit]))
            init.add_clause(tuple(walk_lits))

    def propagate(self, control: PropagateControl, changes: Sequence[int]) -> None:
        assert len(changes) == 1, "Should have been only one change"
        sol_lit = changes[-1]
        walk_lit = self._s2w[sol_lit]
        edge2 = self.__walk_lit_to_edge(walk_lit)
        print("[DEBUG]:", "Reached fixpoint, chose {}".format(edge2))






In [6]:
windy_propagator = WindyPropagator()

In [7]:
ctl = clingo.Control()
ctl.configuration.solve.models = 0

In [8]:
ctl.register_propagator(windy_propagator)

In [9]:
ctl.add('base', (), inst1)
ctl.add('base', (), reasoning)

In [10]:
ctl.ground([('base', [])])


In [16]:
sorted(ctl.configuration.keys)

['asp',
 'configuration',
 'learn_explicit',
 'parse_ext',
 'parse_maxsat',
 'sat_prepro',
 'share',
 'solve',
 'solver',
 'stats',
 'tester']

In [17]:
sorted(ctl.configuration.solve.keys)

['distribute',
 'enum_mode',
 'global_restarts',
 'integrate',
 'models',
 'opt_mode',
 'parallel_mode',
 'project',
 'solve_limit']

In [20]:
ctl.configuration.solve.parallel_mode

'1,compete'

In [21]:
ctl.configuration.solve.distribute

'conflict,global,4,4194303'

In [ ]:
def get_time(atom: clingo.Symbol):
    if len(atom.arguments) == 2:
        if atom.arguments[-1].type is clingo.SymbolType.Number:
            return atom.arguments[-1].number
    return float('Inf')

In [ ]:
print("[DEBUG]:", "Starting to solve")
with ctl.solve(yield_=True) as solve_handle:
    print("[DEBUG]:", "Finished solving")
    for model in solve_handle:
        atoms = model.symbols(shown=True)
        print("Answer {}: {} {} {}".format(model.number, '{', ' '.join(map(str, sorted(sorted(atoms), key=get_time))),
                                           '}'))
    print(solve_handle.get())

In [ ]:
print("Done")